In [2]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('player_stats.csv')
df

,Unnamed: 0,match_id,game_id,team,score_team,opponent,score_opp,win_lose,map,map_pick,...,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd
0,0,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,1.19,239,15,11,6,72%,146,16%,1,0
1,1,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.84,234,15,14,6,61%,145,32%,1,4
2,2,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.68,173,10,15,2,67%,123,18%,4,2
3,3,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.63,142,8,15,4,83%,92,29%,3,2
4,4,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.57,114,7,16,8,61%,63,13%,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,6225,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.43,244,23,13,10,83%,162,46%,0,1
6226,6226,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.26,338,26,18,5,78%,225,18%,7,3
6227,6227,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,1.09,198,16,16,1,65%,131,36%,2,2
6228,6228,61192,137642,EG,13,PRX,10,team win,Lotus,team pick,...,0.86,180,13,17,15,74%,108,29%,2,3


In [49]:
df.dtypes

Unnamed: 0     int64
match_id       int64
game_id        int64
team          object
score_team     int64
opponent      object
score_opp      int64
win_lose      object
map           object
map_pick      object
player_id      int64
player        object
agent         object
rating        object
acs           object
kill           int64
death          int64
assist         int64
kast%         object
adr           object
hs%           object
fk            object
fd            object
dtype: object

In [57]:
def to_float(string):
    if string.strip() == '':
        return 0.0
    if '%' in string:
        return float(string[:-1].strip())
    return float(string.strip())

In [68]:
conv_columns = ['rating','acs','kast%','adr','hs%','fk','fd']
for col in conv_columns:
    df = df.assign(col = df[col].apply(to_float))

In [60]:
# encoding binary data
df_new = df.drop(columns = ['match_id','Unnamed: 0','game_id'])
df_new = df_new.assign(team_win = df['win_lose']=='team win').drop(columns=['win_lose'])
df_new = df_new.assign(map_pick = df_new['map_pick'] == 'team pick')

In [61]:
player_dict = {}
for i in df_new['player_id'].unique():
    player_dict[i] = df_new[df_new['player_id'] == i]['player'].iloc[0]
np.save('player_dict.npy', player_dict)

In [62]:
df_new['agent'].unique()
agents = {'initiator': ['sova','skye','kayo','fade','breach','gekko'],
               'sentinel': ['killjoy','cypher','sage','chamber'],
               'controller': ['omen','viper','harbor','astra','brimstone'],
               'duelist': ['jett','raze','phoenix','yoru','neon','reyna']
               }
agent_class = {}
for key,lst in agents.items():
    for a in lst:
        agent_class[a] = key

agent_class

{'sova': 'initiator',
 'skye': 'initiator',
 'kayo': 'initiator',
 'fade': 'initiator',
 'breach': 'initiator',
 'gekko': 'initiator',
 'killjoy': 'sentinel',
 'cypher': 'sentinel',
 'sage': 'sentinel',
 'chamber': 'sentinel',
 'omen': 'controller',
 'viper': 'controller',
 'harbor': 'controller',
 'astra': 'controller',
 'brimstone': 'controller',
 'jett': 'duelist',
 'raze': 'duelist',
 'phoenix': 'duelist',
 'yoru': 'duelist',
 'neon': 'duelist',
 'reyna': 'duelist'}

In [63]:
df_new = df_new.assign(agent_class = df_new['agent'].apply(lambda x: agent_class[x]))
df_new = df_new.drop(columns=['agent','player'])
df_new

,team,score_team,opponent,score_opp,map,map_pick,player_id,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd,team_win,agent_class
0,ZETA,5,DRX,13,Ascent,False,999,1.19,239,15,11,6,72%,146,16%,1,0,False,initiator
1,ZETA,5,DRX,13,Ascent,False,1395,0.84,234,15,14,6,61%,145,32%,1,4,False,sentinel
2,ZETA,5,DRX,13,Ascent,False,424,0.68,173,10,15,2,67%,123,18%,4,2,False,duelist
3,ZETA,5,DRX,13,Ascent,False,1000,0.63,142,8,15,4,83%,92,29%,3,2,False,initiator
4,ZETA,5,DRX,13,Ascent,False,6668,0.57,114,7,16,8,61%,63,13%,0,1,False,controller
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,EG,13,PRX,10,Lotus,True,26171,1.43,244,23,13,10,83%,162,46%,0,1,True,controller
6226,EG,13,PRX,10,Lotus,True,3993,1.26,338,26,18,5,78%,225,18%,7,3,True,duelist
6227,EG,13,PRX,10,Lotus,True,604,1.09,198,16,16,1,65%,131,36%,2,2,True,sentinel
6228,EG,13,PRX,10,Lotus,True,11225,0.86,180,13,17,15,74%,108,29%,2,3,True,initiator


In [64]:
df_numerical = pd.get_dummies(df_new, columns=['map','agent_class','team','opponent'])
df_numerical.to_csv('numerical_data.csv')

In [21]:
df_player = df_new.drop(columns = ['team','score_team','opponent','score_opp'])
df_player.to_csv('player_preprocessed.csv')

In [30]:
df_team = df.drop(columns = ['match_id','Unnamed: 0','agent','player_id','player'])
df_team = df_team.assign(team_win = df['win_lose']=='team win').drop(columns=['win_lose'])
df_team = df_team.assign(map_pick = df_team['map_pick'] == 'team pick')

,game_id,team,score_team,opponent,score_opp,map,map_pick,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd,team_win
0,119512,ZETA,5,DRX,13,Ascent,False,1.19,239,15,11,6,72%,146,16%,1,0,False
1,119512,ZETA,5,DRX,13,Ascent,False,0.84,234,15,14,6,61%,145,32%,1,4,False
2,119512,ZETA,5,DRX,13,Ascent,False,0.68,173,10,15,2,67%,123,18%,4,2,False
3,119512,ZETA,5,DRX,13,Ascent,False,0.63,142,8,15,4,83%,92,29%,3,2,False
4,119512,ZETA,5,DRX,13,Ascent,False,0.57,114,7,16,8,61%,63,13%,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6225,137642,EG,13,PRX,10,Lotus,True,1.43,244,23,13,10,83%,162,46%,0,1,True
6226,137642,EG,13,PRX,10,Lotus,True,1.26,338,26,18,5,78%,225,18%,7,3,True
6227,137642,EG,13,PRX,10,Lotus,True,1.09,198,16,16,1,65%,131,36%,2,2,True
6228,137642,EG,13,PRX,10,Lotus,True,0.86,180,13,17,15,74%,108,29%,2,3,True


In [42]:
def to_percent(string):
    substr = string.remove('%')
    percent = int(substr)
    return percent / 100

In [44]:
df_team.assign(kast = df_team['kast%'].apply(to_percent))

AttributeError: 'str' object has no attribute 'remove'

game_id        int64
team          object
score_team     int64
opponent      object
score_opp      int64
map           object
map_pick        bool
rating        object
acs           object
kill           int64
death          int64
assist         int64
kast%         object
adr           object
hs%           object
fk            object
fd            object
team_win        bool
dtype: object